Adding more provider entries to the data from the previous scrape 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import re
import time

`SELECT [CMS_Certification_Number] FROM [ProviderDataRegistry].[dbo].[FacilityProfile]`

In [6]:
every_ID = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\every_provider.csv")["CMS_Certification_Number"]
previous_IDs = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\Federal_provider_numbers.csv")["Federal_Provider_Number"]

IDs = []
for i in every_ID:
    if i not in previous_IDs:
        IDs.append(str(i))

print(len(IDs))


29116


Looking at the length, it would appear that the values in every_ID are all new. 

In [17]:
try:
    driver.close()
except Exception:
    pass

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://webpricer.cms.gov/#/provider-directory/ipps')
driver.find_element(by = By.XPATH, value = "/html/body/div[3]/div/div/div/div/aside/a[2]").send_keys(Keys.RETURN)

for i in range(len(IDs)):
    while(len(IDs[i]) < 6):  
        IDs[i] = "0" + IDs[i]
     
final_list = []
df = pd.DataFrame([])
count = 1
l = len(IDs)
bad_ID = []

for ID in IDs:
    driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").send_keys(ID)
    time.sleep(1)

    # Since this data has provider IDs that dont return a match, the program logs and skips an ID if it does not get a response within 10 seconds
    stop = False
    while True:
        try:
            table1 = driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[1]/table/tbody").text
            stop = False
            break
        except Exception:
            if stop:
                break
            time.sleep(10)
            stop = True
    if stop:
        bad_ID.append(ID)
        stop = False
        count += 1
        driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").clear()
        continue

    stop = False
    while True:
        try:
            table2 = driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[2]/table/tbody").text
            stop = False
            break
        except Exception:
            if stop:
                break
            time.sleep(10)
            stop = True
    if stop:
        bad_ID.append(ID)
        stop = False
        count += 1
        driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").clear()
        continue

    table = table1 + "\n" + table2
    provider_list = [ID]
    for i in table.split("\n"):
        provider_list.append(i[i.rfind(' ')+1:])
    final_list.append(provider_list)
    driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").clear()
    print("\ (•◡•) /   Extracted data from Provider ID: " + ID + "!   \ (•◡•) /", end = "  ")
    print("  <<"+str(count)+" out of "+str(l)+" provider's info taken!>>", end = "\r")
    count += 1
    
column_names = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\CMS_provider_directory_all.csv").columns
df = pd.DataFrame(final_list, columns= column_names)
df.head(10)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\email\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=102.0.5005.115)
Stacktrace:
Backtrace:
	Ordinal0 [0x0120D953+2414931]
	Ordinal0 [0x0119F5E1+1963489]
	Ordinal0 [0x0108C6B8+837304]
	Ordinal0 [0x01074120+737568]
	Ordinal0 [0x010D6E29+1142313]
	Ordinal0 [0x010E3FB2+1195954]
	Ordinal0 [0x010D42B6+1131190]
	Ordinal0 [0x010AE860+976992]
	Ordinal0 [0x010AF756+980822]
	GetHandleVerifier [0x0147CC62+2510274]
	GetHandleVerifier [0x0146F760+2455744]
	GetHandleVerifier [0x0129EABA+551962]
	GetHandleVerifier [0x0129D916+547446]
	Ordinal0 [0x011A5F3B+1990459]
	Ordinal0 [0x011AA898+2009240]
	Ordinal0 [0x011AA985+2009477]
	Ordinal0 [0x011B3AD1+2046673]
	BaseThreadInitThunk [0x76986739+25]
	RtlGetFullPathName_UEx [0x77B38FEF+1215]
	RtlGetFullPathName_UEx [0x77B38FBD+1165]


In [16]:
column_names = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\CMS_provider_directory_all.csv").columns[1:]
df = pd.DataFrame(final_list, columns= column_names)
df.head(10)

,ProviderID,Fiscal_year_begin_date,Report_date,Termination_Date,Waiver_code,Intermediary_number,Provider_type,Current_Census_division,Change_code_for_reclass,Actual_Geographic_MSA,...,Hospital_readmission_reduction_adjustment,Model_1_bundle_percent,Hospital_acquired_condition_(HAC)_reduction_program_participant,Uncompensated_care_amount,Electronic_health_record_(EHR)_incentive_program_reduction_indicator,Low-volume_adjustment_factor,County_code,State_code,Medicare_performance_adjustment,LTCH_DPP_indicator
0,520076,07/01/2021,03/31/2022,--,N,06001,07,4,Y,52,...,0.9886,0,N,"$1,178.53",--,0.176367,55027,52,0,--
1,170200,08/01/2021,03/31/2022,--,N,05001,00,6,--,9040,...,0.9909,0,Y,$0.00,--,0,20173,17,0,--
2,370039,11/01/2021,03/31/2022,--,N,05901,00,7,--,8560,...,0.9968,0,N,$904.78,--,0.089167,40131,37,0,--
3,140013,01/01/2022,03/31/2022,--,N,06101,00,4,N,6120,...,0.9964,0,N,$0.00,--,0,17143,14,0,--
4,451350,05/01/2016,03/31/2022,--,Y,04011,37,7,N,45,...,0,0,--,$0.00,--,0,--,45,0,--
5,450677,01/01/2022,03/31/2022,--,N,09101,00,7,N,2800,...,0.9923,0,N,"$1,382.67",--,0,48439,45,0,--
6,240019,07/01/2021,03/31/2022,--,N,06201,00,6,--,2240,...,1,0,N,$609.02,Y,0,27137,24,0,--
7,360003,07/01/2021,03/31/2022,--,N,15201,00,4,N,1640,...,0.9996,0,Y,"$1,006.36",--,0,39061,36,0,--
8,460051,07/01/2021,03/31/2022,--,N,03001,00,8,N,7160,...,0.9852,0,N,"$2,363.83",--,0,49035,46,0,--
9,450537,07/01/2021,03/31/2022,--,N,04011,00,7,N,1920,...,0.9904,0,N,"$1,392.25",--,0,48113,45,0,--


Chrome ran out of memory, restarting from where it left off:

In [18]:
print(count)

10306


count has a value 10306, there it has exited the loop 10305 times and must start from the 10306th element

Adding condition to restart the browser every 1000 iterations to prevent running out of memory again

In [20]:
try:
    driver.close()
except Exception:
    pass

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://webpricer.cms.gov/#/provider-directory/ipps')
driver.find_element(by = By.XPATH, value = "/html/body/div[3]/div/div/div/div/aside/a[2]").send_keys(Keys.RETURN)

# for i in range(len(IDs)):
#     while(len(IDs[i]) < 6):  
#         IDs[i] = "0" + IDs[i]
     
# final_list = []
# df = pd.DataFrame([])
count = 10306
l = len(IDs)
# bad_ID = []

IDs = IDs[(count - 1):]

for ID in IDs:
    driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").send_keys(ID)
    time.sleep(1)

    # Since this data has provider IDs that dont return a match, the program logs and skips an ID if it does not get a response within 10 seconds
    stop = False
    while True:
        try:
            table1 = driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[1]/table/tbody").text
            stop = False
            break
        except Exception:
            if stop:
                break
            time.sleep(6)
            stop = True
    if stop:
        bad_ID.append(ID)
        stop = False
        count += 1
        driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").clear()
        continue

    stop = False
    while True:
        try:
            table2 = driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[2]/table/tbody").text
            stop = False
            break
        except Exception:
            if stop:
                break
            time.sleep(6)
            stop = True
    if stop:
        bad_ID.append(ID)
        stop = False
        count += 1
        driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").clear()
        continue

    table = table1 + "\n" + table2
    provider_list = [ID]
    for i in table.split("\n"):
        provider_list.append(i[i.rfind(' ')+1:])
    final_list.append(provider_list)
    driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").clear()
    print("\ (•◡•) /   Extracted data from Provider ID: " + ID + "!   \ (•◡•) /", end = "  ")
    print("  <<"+str(count)+" out of "+str(l)+" provider's info taken!>>", end = "\r")

    if (count % 1000) == 000:
        driver.close()
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.get('https://webpricer.cms.gov/#/provider-directory/ipps')
        time.sleep(2)
        driver.find_element(by = By.XPATH, value = "/html/body/div[3]/div/div/div/div/aside/a[2]").send_keys(Keys.RETURN)

    count += 1
    
column_names = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\CMS_provider_directory_all.csv").columns
df = pd.DataFrame(final_list, columns= column_names)
df.head(10)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\email\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


\ (•◡•) /   Extracted data from Provider ID: 175078!   \ (•◡•) /    <<11000 out of 29116 provider's info taken!>>


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\email\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


\ (•◡•) /   Extracted data from Provider ID: 275025!   \ (•◡•) /    <<12000 out of 29116 provider's info taken!>>


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\email\.wdm\drivers\chromedriver\win32\102.0.5005.61]


...

\ (•◡•) /   Extracted data from Provider ID: 285203!   \ (•◡•) /    <<28000 out of 29116 provider's info taken!>>


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\email\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


\ (•◡•) /   Extracted data from Provider ID: 235349!   \ (•◡•) /    <<29000 out of 29116 provider's info taken!>>


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\email\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


ValueError: 62 columns passed, passed data had 61 columns

Column name list contains extra column added because of the number column that pandas adds while saving, simply remove its first element and it works fine 

In [21]:
column_names = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\CMS_provider_directory_all.csv").columns[1:]
df = pd.DataFrame(final_list, columns= column_names)
df.head(10)

,ProviderID,Fiscal_year_begin_date,Report_date,Termination_Date,Waiver_code,Intermediary_number,Provider_type,Current_Census_division,Change_code_for_reclass,Actual_Geographic_MSA,...,Hospital_readmission_reduction_adjustment,Model_1_bundle_percent,Hospital_acquired_condition_(HAC)_reduction_program_participant,Uncompensated_care_amount,Electronic_health_record_(EHR)_incentive_program_reduction_indicator,Low-volume_adjustment_factor,County_code,State_code,Medicare_performance_adjustment,LTCH_DPP_indicator
0,520076,07/01/2021,03/31/2022,--,N,06001,07,4,Y,52,...,0.9886,0,N,"$1,178.53",--,0.176367,55027,52,0,--
1,170200,08/01/2021,03/31/2022,--,N,05001,00,6,--,9040,...,0.9909,0,Y,$0.00,--,0,20173,17,0,--
2,370039,11/01/2021,03/31/2022,--,N,05901,00,7,--,8560,...,0.9968,0,N,$904.78,--,0.089167,40131,37,0,--
3,140013,01/01/2022,03/31/2022,--,N,06101,00,4,N,6120,...,0.9964,0,N,$0.00,--,0,17143,14,0,--
4,451350,05/01/2016,03/31/2022,--,Y,04011,37,7,N,45,...,0,0,--,$0.00,--,0,--,45,0,--
5,450677,01/01/2022,03/31/2022,--,N,09101,00,7,N,2800,...,0.9923,0,N,"$1,382.67",--,0,48439,45,0,--
6,240019,07/01/2021,03/31/2022,--,N,06201,00,6,--,2240,...,1,0,N,$609.02,Y,0,27137,24,0,--
7,360003,07/01/2021,03/31/2022,--,N,15201,00,4,N,1640,...,0.9996,0,Y,"$1,006.36",--,0,39061,36,0,--
8,460051,07/01/2021,03/31/2022,--,N,03001,00,8,N,7160,...,0.9852,0,N,"$2,363.83",--,0,49035,46,0,--
9,450537,07/01/2021,03/31/2022,--,N,04011,00,7,N,1920,...,0.9904,0,N,"$1,392.25",--,0,48113,45,0,--


In [24]:
print(len(bad_ID))
print(len(final_list))
print(len(bad_ID) + len(final_list))


4147
24969
29116


In [27]:
for i in range(len(final_list)):
    for j in range(len(final_list[i])):
        if final_list[i][j] == "--":
            final_list[i][j] = ""

In [28]:
df = pd.DataFrame(final_list, columns= column_names)
df.head(10)

,ProviderID,Fiscal_year_begin_date,Report_date,Termination_Date,Waiver_code,Intermediary_number,Provider_type,Current_Census_division,Change_code_for_reclass,Actual_Geographic_MSA,...,Hospital_readmission_reduction_adjustment,Model_1_bundle_percent,Hospital_acquired_condition_(HAC)_reduction_program_participant,Uncompensated_care_amount,Electronic_health_record_(EHR)_incentive_program_reduction_indicator,Low-volume_adjustment_factor,County_code,State_code,Medicare_performance_adjustment,LTCH_DPP_indicator
0,520076,07/01/2021,03/31/2022,,N,06001,07,4,Y,52,...,0.9886,0,N,"$1,178.53",,0.176367,55027,52,0,
1,170200,08/01/2021,03/31/2022,,N,05001,00,6,,9040,...,0.9909,0,Y,$0.00,,0,20173,17,0,
2,370039,11/01/2021,03/31/2022,,N,05901,00,7,,8560,...,0.9968,0,N,$904.78,,0.089167,40131,37,0,
3,140013,01/01/2022,03/31/2022,,N,06101,00,4,N,6120,...,0.9964,0,N,$0.00,,0,17143,14,0,
4,451350,05/01/2016,03/31/2022,,Y,04011,37,7,N,45,...,0,0,,$0.00,,0,,45,0,
5,450677,01/01/2022,03/31/2022,,N,09101,00,7,N,2800,...,0.9923,0,N,"$1,382.67",,0,48439,45,0,
6,240019,07/01/2021,03/31/2022,,N,06201,00,6,,2240,...,1,0,N,$609.02,Y,0,27137,24,0,
7,360003,07/01/2021,03/31/2022,,N,15201,00,4,N,1640,...,0.9996,0,Y,"$1,006.36",,0,39061,36,0,
8,460051,07/01/2021,03/31/2022,,N,03001,00,8,N,7160,...,0.9852,0,N,"$2,363.83",,0,49035,46,0,
9,450537,07/01/2021,03/31/2022,,N,04011,00,7,N,1920,...,0.9904,0,N,"$1,392.25",,0,48113,45,0,


In [29]:
df.to_csv("CMS_directory_every_provider.csv")
pd.DataFrame(bad_ID, columns=["Bad_ID"]).to_csv("CMS_directory_every_provider_bad_ids.csv")